In [ ]:
%matplotlib inline

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import math
import pytesseract
import re

from functools import partial

In [ ]:
%run four_point_transform.py

In [ ]:
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

In [ ]:
def draw(img):
    plt.figure(figsize=(10, 10))
    plt.imshow(img, cmap='gray')
    
def Gray(img):
    return cv.cvtColor(img, cv.COLOR_BGR2GRAY)

def TopHat(img, size=(9, 9)):
    kernel = cv.getStructuringElement(cv.MORPH_RECT, size)
    return cv.morphologyEx(img, cv.MORPH_TOPHAT, kernel)

def Open(img, size=(9, 9)):
    kernel = cv.getStructuringElement(cv.MORPH_RECT, size)
    return cv.morphologyEx(img, cv.MORPH_OPEN, kernel)

def Close(img, size=(9, 9)):
    kernel = cv.getStructuringElement(cv.MORPH_RECT, size)
    return cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)

def AdaptiveThreshold(img):
    thresh = cv.adaptiveThreshold(img, 255, 
                                  cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                  thresholdType=cv.THRESH_BINARY_INV,
                                  blockSize=9,
                                  C=5)
    return thresh

def Otsu(img):
    thresh = cv.threshold(img, 0, 255, cv.THRESH_BINARY|cv.THRESH_OTSU)[1]
    return thresh

def Inverse(img):
    return cv.bitwise_not(img)

def Canny(img):
    return cv.Canny(thresh, 50, 200, None, 3)

def Dilate(img, size=(3, 3), kernel=None):
    if kernel is None:
        kernel = cv.getStructuringElement(cv.MORPH_RECT, size)
    return cv.dilate(img, kernel)

def Erode(img, size=(5, 5), kernel=None):
    if kernel is None:
        kernel = cv.getStructuringElement(cv.MORPH_RECT, size)
    return cv.erode(img, kernel)

# nonzeros = cv.findNonZero(thresh)

def MinAreaRect(contour):
    rect = cv.minAreaRect(contour)
    box = cv.boxPoints(rect)
    cnt = np.int0(box)
    return cnt

# vis = cv.drawContours(img.copy(), [cnt], -1, (0, 255, 0), 1)
# draw(vis)

def FindOuterContour(thresh):
    cnts, _ = cv.findContours(thresh,
                              cv.RETR_LIST,
                              cv.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, 
                  key=cv.contourArea, 
                  reverse=True)
    cnt = None
    for c in cnts:
        # Approximate the contour.
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            cnt = approx
            break
    return cnt

def DrawContour(img, cnts, color=GREEN, width=3):
    draw(cv.drawContours(img.copy(), cnts, -1, color, width))
    
# def sobelxy(img, ksize=5):
#     sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize)  # x
#     sobely = cv.Sobel(sobelx, cv.CV_64F, 0, 1, ksize)  # y
#     return sobely

def HoughLines(src, color=GREEN, width=2, threshold=200):
    # rho in pixels.
    rho = 1

    # theta in degree.
    theta = np.pi/180

    # The minimum number of intersections to detect a line.
#     threshold = 200

    lines = cv.HoughLines(src, rho, theta, threshold, None, 0, 0)
    if lines is not None:
        for rho, theta in lines[:, 0]:
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))

            cv.line(src, pt1, pt2, color, width, cv.LINE_AA)
    return src


def HoughLinesP(img, color=(255, 255, 255), width=1):
    minLineLength = img.shape[1] * 0.2
    maxLineGap = 10
    lines = cv.HoughLinesP(img, 1, np.pi/180, 100, minLineLength, maxLineGap)
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv.line(img, (x1,y1), (x2,y2), color, width)
    return img

def Laplacian(img):
    laplacian = cv.Laplacian(img, cv.CV_8UC1) # Laplacian Edge Detection
    return laplacian

def Sobel(src):    
    # Sobel on the x-axis helps remove vertical line.
    grad_x = cv.Sobel(src, ddepth=cv.CV_32F, dx=1, dy=0, ksize=-1)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_x = cv.normalize(abs_grad_x, abs_grad_x, 255, 0)

    # Sobel on the y-axis helps remove vertical line.
    grad_y = cv.Sobel(src, ddepth=cv.CV_32F, dx=0, dy=1, ksize=-1)
    abs_grad_y = cv.convertScaleAbs(grad_y)    
    abs_grad_y = cv.normalize(abs_grad_y, abs_grad_y, 255, 0)

    sobel = cv.bitwise_and(abs_grad_x, abs_grad_y)
    return cv.bitwise_and(sobel, src)

def GaussianBlur(src, size=(3, 3)):
    return cv.GaussianBlur(src, size, -1)

In [ ]:
def match_int(s):
    match = re.match('\d+', s)
    if match is None:
        return -1
    return int(match.group(0))

In [ ]:
# src = 'sudoku.jpeg'
# src = 'sudoku_1.png'
# src = 'sudoku_2.png'
src = 'sudoku_3.png'

img = cv.imread(src)

draw(img)

In [ ]:
class Pipeline:
    def __init__(self, pipelines):
        self.pipelines = pipelines
    def exec(self, src):
        dst = src.copy()
        self.steps = {}
        for name, fn in self.pipelines:
            dst = fn(dst)
            self.steps[name] = dst.copy()
        return dst

In [ ]:
height, width = img.shape[:2]

# Assuming user takes the full photo of sudoku - so the whole grid fits into
# the picture. Since it's a square, take the minimum dimension. 
# Assume that the box fit to at least 80% of the image (the Pareto's Principle).
dim = min(height, width) * 0.8

# Divide it by 9 since one row/column will have 9 cells. 
# Use that as the kernel size.
cell = int(dim / 9) # Equivalent to math.ceil.

pipeline = Pipeline([('gray', Gray),
                     ('inverse', Inverse),
                     ('blur', GaussianBlur),
                     # We want to use tophat to capture only the grid - 
                     # anything larger than it will be excluded.
                     ('tophat', partial(TopHat, size=(cell, cell))),
                     ('thresh', Otsu),
                    ])

dst = pipeline.exec(img)
draw(dst)
cnt = FindOuterContour(dst)
# DrawContour(dst, [cnt])

In [ ]:
transform = four_point_transform(dst, cnt.reshape(4, 2))
draw(transform)

In [ ]:
# Take full dim now that we have transformed the image to fit full screen.
height, width = transform.shape[:2]
dim = min(height, width) * 0.8


# Divide it by 9 since one row/column will have 9 cells. 
# Use that as the kernel size.
cell = int(dim / 9) # Equivalent to math.ceil.
font = int(cell / 3) # Assume the font is just 33% of the cell width.

print('dim', dim)
print('cell', cell)
print('font', font)

pipeline2 = Pipeline([('hough', partial(HoughLines, color=BLACK, width=3, threshold=200)),
                      ('tophat', partial(TopHat, size=(cell, cell))),
                      ('sobel', Sobel),
                      ('close', partial(Close, size=(font, font))),
                      ('dilate', partial(Dilate, size=(font, font))),
                      ('thresh', Otsu)
])
out = pipeline2.exec(transform)
draw(out)

In [ ]:
tgt = Inverse(transform)

cnts, _ = cv.findContours(out, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
cnts = [c for c in cnts
        if cv.contourArea(c) > cell*cell/4]
# grid = [MinAreaRect(c) 
#         for c in cnts]

vis = tgt.copy()
for c in cnts:
    x, y, w, h = cv.boundingRect(c)
    cv.rectangle(vis, (x, y),(x + w, y + h), GREEN, 1)

print(len(cnts))
draw(vis)

In [ ]:
%%time

# Remove border.
rois = []
height, width = tgt.shape[:2]

# Additional padding which is the border.
p = 1

sudoku = np.zeros((9, 9), np.int)

for c in cnts:
    x, y, w, h = cv.boundingRect(c)
    # The region of interest.
    roi = tgt[max(0, y-p):min(y+h+p, height), max(0, x-p):min(x+w+p, width)]
    
    # Set edges to white.
    roi[:, 0] = 255
    roi[:, -1] = 255
    roi[0, :] = 255
    roi[-1, :] = 255
    rois.append(roi)
    
    text = pytesseract.image_to_string(roi, config='--oem 1 --psm 6') # Needs to be enabled to read single digit.
    out = match_int(text)
    if out == -1: continue

    mid_x = x + (w / 2)
    mid_y = y + (h / 2)
    i, j = int(mid_y/height * 9), int(mid_x/width * 9)
    sudoku[i, j] = out

In [ ]:
len(sudoku[sudoku > 0])

In [ ]:
sudoku

In [ ]:
i = 0

In [ ]:
draw(rois[i])
i += 1

In [ ]:
# vis = transform.copy()


# draw(vis)

In [ ]:
sudoku = np.array([[0, 0, 0, 0, 0, 4, 0, 9, 0],
                   [8, 0, 2, 9, 7, 0, 0, 0, 0],
                   [9, 0, 1, 2, 0, 0, 3, 0, 0],
                   [0, 0, 0, 0, 4, 9, 1, 5, 7],
                   [0, 1, 3, 0, 5, 0, 9, 2, 0],
                   [5, 7, 9, 1, 2, 0, 0, 0, 0],
                   [0, 0, 7, 0, 0, 2, 6, 0, 3],
                   [0, 0, 0, 0, 3, 8, 2, 0, 5],
                   [0, 2, 0, 5, 0, 0, 0, 0, 0]])
# sudoku = np.array([[7, 3, 5, 6, 1, 4, 8, 9, 2],
#                    [8, 4, 2, 9, 7, 3, 5, 6, 1],
#                    [9, 6, 1, 2, 8, 5, 3, 7, 4],
#                    [2, 8, 6, 3, 4, 9, 1, 5, 7],
#                    [4, 1, 3, 8, 5, 7, 9, 2, 6],
#                    [5, 7, 9, 1, 2, 6, 4, 3, 8],
#                    [1, 5, 7, 4, 9, 2, 6, 8, 3],
#                    [6, 9, 4, 7, 3, 8, 2, 1, 5],
#                    [3, 2, 8, 5, 6, 1, 7, 4, 9]])

In [ ]:
class Node:
    def __init__(self, L=None, R=None, U=None, D=None):
        self.L = L # Left.
        self.R = R # Right.
        self.U = U # Up.
        self.D = D # Down.

# http://taeric.github.io/Sudoku.html
# https://www.ocf.berkeley.edu/~jchu/publicportal/sudoku/0011047.pdf
class ListHeader(Node):
    def __init__(self, L=None, R=None, U=None, D=None, C=None, N=None, S=None):
        self.N = N # Names.
        self.S = S # Sizes.
        self.C = C # Column Object.
        
        Node.__init__(self, L, R, U, D)

alphabets = list('abcdefghijklmnopqrstuvwxyz')

# Setup root header.
root = ListHeader(N='root')
rows, cols = sudoku.shape

# Link all links header to root.
node = root
for c in range(cols):
    node.R = ListHeader(N=alphabets[c], L=node)
    node = node.R
node.R = root
root.L = node